# ChatFlow Example

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, json, copy

from colink import CoLink

from aiflows.utils import serve_utils
from aiflows.utils.general_helpers import read_yaml_file
from aiflows.messages import FlowMessage
from aiflows.utils import coflows_utils, colink_utils
from aiflows.workers import run_dispatch_worker_threads, run_dispatch_worker_thread
from aiflows import flow_verse

dependencies = [
    {"url": "aiflows/ChatFlowModule", "revision": "coflows"},
    {"url": "aiflows/HumanStandardInputFlowModule", "revision": "coflows"},
    {"url": "aiflows/ChatInteractiveFlowModule", "revision": "coflows"}
]
flow_verse.sync_dependencies(dependencies)

[2024-03-17 07:10:22,191][aiflows.flow_verse.loading:775][INFO] - [<interactive>] started to sync flow module dependencies to /home/staverm/workspace/coflows-dev/aiflows/examples/mauro_examples/InteractiveChatFlow/flow_modules...
[2024-03-17 07:10:22,325][aiflows.flow_verse.loading:608][INFO] - aiflows/ChatFlowModule:coflows already synced, skip
[2024-03-17 07:10:22,416][aiflows.flow_verse.loading:608][INFO] - aiflows/HumanStandardInputFlowModule:coflows already synced, skip
[2024-03-17 07:10:22,511][aiflows.flow_verse.loading:608][INFO] - aiflows/ChatInteractiveFlowModule:coflows already synced, skip
[2024-03-17 07:10:22,512][aiflows.flow_verse.loading:825][INFO] - [<interactive>] finished syncing




['/home/staverm/workspace/coflows-dev/aiflows/examples/mauro_examples/InteractiveChatFlow/flow_modules/aiflows/ChatInteractiveFlowModule',
 '/home/staverm/workspace/coflows-dev/aiflows/examples/mauro_examples/InteractiveChatFlow/flow_modules/aiflows/ChatFlowModule',
 '/home/staverm/workspace/coflows-dev/aiflows/examples/mauro_examples/InteractiveChatFlow/flow_modules/aiflows/HumanStandardInputFlowModule']

# Connect to CoLink Server

In [5]:
# This example only works with a server outside of the notebook because we will need to also start the human UI in a separate terminal
USER_JWT_PATH = "/home/staverm/workspace/coflows-dev/coflows/jwts.txt"
user_jwt = open(USER_JWT_PATH).readline().strip()
cl = CoLink("http://127.0.0.1:2021", user_jwt)

# Start a few default workers

In [6]:
run_dispatch_worker_thread(cl)
run_dispatch_worker_thread(cl)

Dispatch worker started in attached thread.
dispatch_point: coflows_dispatch
Dispatch worker started in attached thread.
dispatch_point: coflows_dispatch


ERROR:root:CreateEntry Received RPC exception: code=StatusCode.INTERNAL message=Key name already exists: 0321b9d6b173e80fb16e0e4be99555b3ca3674e108c8a2736a246c1b345c4184c7::_lock:_internal:protocols:coflows_dispatch:local:operator_mq
ERROR:root:CreateEntry Received RPC exception: code=StatusCode.INTERNAL message=Key name already exists: 0321b9d6b173e80fb16e0e4be99555b3ca3674e108c8a2736a246c1b345c4184c7::_lock:_internal:protocols:coflows_dispatch:local:operator_mq
ERROR:root:CreateEntry Received RPC exception: code=StatusCode.INTERNAL message=Key name already exists: 0321b9d6b173e80fb16e0e4be99555b3ca3674e108c8a2736a246c1b345c4184c7::_lock:_internal:protocols:coflows_dispatch:local:operator_mq



~~~ Dispatch task ~~~
flow_endpoint: InteractiveChat
flow_id: d63ce2fe-70b4-42fb-9f75-9bd432a55f0b
owner_id: local
message_paths: ['push_tasks:20453d4c-f0ea-4bff-a12c-56105ee5a1db:msg']
parallel_dispatch: False

Input message source: unknown

~~~ Dispatch task ~~~
flow_endpoint: Assistant
flow_id: 2d3fe7ef-653e-43d1-a5ff-4572d54d5158
owner_id: local
message_paths: ['push_tasks:52ad2f77-814f-40f3-87c5-267ec4d8e171:msg']
parallel_dispatch: False

Input message source: Assistant
CHATFLOW YO, input: {'id': 0, 'query': 'I want to ask you a few questions', 'answer': None, 'end_of_interaction': False}

~~~ Dispatch task ~~~
flow_endpoint: Assistant
flow_id: 37d39b48-a205-495c-923c-fe49572a96da
owner_id: local
message_paths: ['push_tasks:164a3ec9-4a3c-471d-a702-b30e2e45b6a4:msg']
parallel_dispatch: False

Input message source: Assistant
CHATFLOW YO, input: {'id': 0, 'query': 'I want to ask you a few questions. Please ONLY respond in Croatian.', 'answer': None, 'end_of_interaction': False}
CHA

In [7]:
run_dispatch_worker_thread(cl)

Dispatch worker started in attached thread.
dispatch_point: coflows_dispatch

~~~ Dispatch task ~~~
flow_endpoint: InteractiveChat
flow_id: ca17ee52-db8d-4b67-96d2-90fae867f388
owner_id: local
message_paths: ['push_tasks:0114eace-a06e-40f0-ab20-4365a7fa5250:msg']
parallel_dispatch: False

Input message source: unknown

~~~ Dispatch task ~~~
flow_endpoint: InteractiveChat
flow_id: d63ce2fe-70b4-42fb-9f75-9bd432a55f0b
owner_id: local
message_paths: ['push_tasks:2dec2664-e2ea-48a0-a8ac-8633e782dda2:msg']
parallel_dispatch: False

Input message source: Assistant

~~~ Dispatch task ~~~
flow_endpoint: InteractiveChat
flow_id: d63ce2fe-70b4-42fb-9f75-9bd432a55f0b
owner_id: local
message_paths: ['push_tasks:81d8e355-a2e3-4f48-8122-0003af7ffced:msg']
parallel_dispatch: False

Input message source: User

~~~ Dispatch task ~~~
flow_endpoint: InteractiveChat
flow_id: ca17ee52-db8d-4b67-96d2-90fae867f388
owner_id: local
message_paths: ['push_tasks:58d84b8e-1985-44c3-94fb-ada29ed4bc98:msg']
parallel

# Serve

In [18]:
serve_utils.serve_flow(
    cl=cl,
    flow_class_name="flow_modules.aiflows.HumanStandardInputFlowModule.HumanStandardInputFlow",
    flow_endpoint="Human",
    singleton=True,
    dispatch_point="human_dispatch",
)
# User should start dispatch_worker with dispatch_point="human_dispatch" in a separate terminal (serves as UI for user)
# plz ensure that you pass the correct flow_modules_base_path when starting worker (give path to flow_modules folder that contains
# HumanStandardInputFlowModule being served here

Already serving at flows:Human


False

In [9]:
serve_utils.serve_flow(
    cl=cl,
    flow_class_name="flow_modules.aiflows.ChatFlowModule.ChatAtomicFlow",
    flow_endpoint="Assistant",
)

Started serving flow_modules.aiflows.ChatFlowModule.ChatAtomicFlow at flows:Assistant.
dispatch_point: coflows_dispatch
parallel_dispatch: False
singleton: False


True

In [10]:
serve_utils.serve_flow(
    cl=cl,
    flow_class_name="flow_modules.aiflows.ChatInteractiveFlowModule.ChatHumanFlowModule",
    flow_endpoint="InteractiveChat",
)

Started serving flow_modules.aiflows.ChatInteractiveFlowModule.ChatHumanFlowModule at flows:InteractiveChat.
dispatch_point: coflows_dispatch
parallel_dispatch: False
singleton: False


True

## Get instance

In [11]:
root_dir = "."
cfg_path = os.path.join(root_dir, "flow_modules/aiflows/ChatInteractiveFlowModule/demo.yaml")
cfg = read_yaml_file(cfg_path)

In [12]:
interactive_chat = serve_utils.get_flow_instance(
    cl=cl,
    flow_endpoint="InteractiveChat",
    config_overrides=cfg,
)

[2024-03-17 07:10:23,296][aiflows.flow_verse.loading:775][INFO] - [flow_modules.aiflows.ChatInteractiveFlowModule] started to sync flow module dependencies to /home/staverm/workspace/coflows-dev/aiflows/examples/mauro_examples/InteractiveChatFlow/flow_modules...
[2024-03-17 07:10:23,387][aiflows.flow_verse.loading:608][INFO] - aiflows/ChatFlowModule:coflows already synced, skip
[2024-03-17 07:10:23,479][aiflows.flow_verse.loading:608][INFO] - aiflows/HumanStandardInputFlowModule:coflows already synced, skip
[2024-03-17 07:10:23,481][aiflows.flow_verse.loading:825][INFO] - [flow_modules.aiflows.ChatInteractiveFlowModule] finished syncing


Mounted 2d3fe7ef-653e-43d1-a5ff-4572d54d5158 at flows:Assistant:mounts:local:2d3fe7ef-653e-43d1-a5ff-4572d54d5158
Mounted b03919e7-3860-4ff1-9f4f-51bb6010f08d at flows:Human:mounts:local:b03919e7-3860-4ff1-9f4f-51bb6010f08d
Mounted d63ce2fe-70b4-42fb-9f75-9bd432a55f0b at flows:InteractiveChat:mounts:local:d63ce2fe-70b4-42fb-9f75-9bd432a55f0b


## Conversation

In [13]:
input_message = FlowMessage(
    data={"id": 0, "query": "I want to ask you a few questions"},
)

future=interactive_chat.send_message(input_message)

In [14]:
interactive_chat_CRO = serve_utils.get_flow_instance(
    cl=cl,
    flow_endpoint="InteractiveChat",
    config_overrides=cfg,
)
input_message = FlowMessage(
    data={"id": 0, "query": "I want to ask you a few questions. Please ONLY respond in Croatian."},
)
future=interactive_chat_CRO.send_message(input_message)

Mounted 37d39b48-a205-495c-923c-fe49572a96da at flows:Assistant:mounts:local:37d39b48-a205-495c-923c-fe49572a96da
Fetched singleton b03919e7-3860-4ff1-9f4f-51bb6010f08d
Mounted ca17ee52-db8d-4b67-96d2-90fae867f388 at flows:InteractiveChat:mounts:local:ca17ee52-db8d-4b67-96d2-90fae867f388


In [15]:
def status():
    colink_utils.print_served_flows(cl, print_values=False)
    print("\nAll flow instances:")
    colink_utils.print_flow_instances(cl)
status()

 Human/
   singleton
   init
   mounts/
     local/
       b03919e7-3860-4ff1-9f4f-51bb6010f08d/
         init
         config_overrides
   flow_class_name
   parallel_dispatch
   default_dispatch_point
 Assistant/
   default_dispatch_point
   flow_class_name
   singleton
   parallel_dispatch
   mounts/
     local/
       37d39b48-a205-495c-923c-fe49572a96da/
         init
         config_overrides
       2d3fe7ef-653e-43d1-a5ff-4572d54d5158/
         config_overrides
         state
         init
   init
 InteractiveChat/
   parallel_dispatch
   flow_class_name
   singleton
   mounts/
     local/
       d63ce2fe-70b4-42fb-9f75-9bd432a55f0b/
         state
         init
         config_overrides
       ca17ee52-db8d-4b67-96d2-90fae867f388/
         state
         init
         config_overrides
   init
   default_dispatch_point

All flow instances:
 ca17ee52-db8d-4b67-96d2-90fae867f388
 37d39b48-a205-495c-923c-fe49572a96da
 2d3fe7ef-653e-43d1-a5ff-4572d54d5158
 b03919e7-3860-4ff1-9f4f-51